In [3]:
import psycopg2

pg_host = "localhost"
pg_database = "demo"
pg_user = "postgres"
pg_password = "postgres"

# Establish PostgreSQL connection
pg_conn = psycopg2.connect(
    host=pg_host,
    database=pg_database,
    user=pg_user,
    password=pg_password
)
pg_cursor = pg_conn.cursor()

In [4]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain, LLMChain

load_dotenv()

True

In [5]:
template1 = """
You are expert software analyst
Sentence 1 : {sentence1}

Sentence 2: {sentence2}

From sentence 1 and sentence 2, is that both refers in same feature of software requirement
Answer must be (yes/no), dont give any other answer than yes or no

"""

prompt1 = ChatPromptTemplate.from_template(template1)

model = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv('OPEN_API_KEY'), temperature=0)

chain1 = prompt1 | model

chain_one = LLMChain(llm=model, prompt=prompt1, output_key="response")

overal_chain = SequentialChain(
    chains=[chain_one],
    input_variables=["sentence1", "sentence2"],
    output_variables=["response"]
)

/var/folders/80/kl6rpl3d2xj20wv4hyb0dbgc0000gn/T/ipykernel_4071/4160468199.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain_one = LLMChain(llm=model, prompt=prompt1, output_key="response")


In [7]:
# Execute query to select similarities
# pg_cursor.execute("""
#     select source_id, ds_source.sentence as source, target_id, ds_target.sentence as target from distinct_label_pair
#     inner join datasets as ds_source on ds_source.id = source_id
#     inner join datasets as ds_target on ds_target.id  = target_id 
#     group by source_id, ds_source.id, target_id, ds_target.id
# """)

pg_cursor.execute("""
select distinct_label_pair.source_id, ds_source.sentence as source, distinct_label_pair.target_id, ds_target.sentence as target from distinct_label_pair
left join prompt_matching on (distinct_label_pair.source_id = prompt_matching.source_id and distinct_label_pair.target_id = prompt_matching.target_id)
inner join datasets as ds_source on ds_source.id = distinct_label_pair.source_id
inner join datasets as ds_target on ds_target.id  = distinct_label_pair.target_id 
where prompt_matching.is_match is null
group by distinct_label_pair.source_id, ds_source.id, distinct_label_pair.target_id, ds_target.id
                  """)
# Fetch all results
distinct = pg_cursor.fetchall()

In [8]:
for data in distinct:
    message = overal_chain({"sentence1": data[1], "sentence2": data[3]})
    response = message['response']
    if response.strip().lower() == 'yes':
        pg_cursor.execute("INSERT INTO prompt_matching (source_id, target_id, model, is_match) VALUES (%s, %s, %s, %s)", (data[0], data[2], 'gpt-4o-mini', True))
    else:
        pg_cursor.execute("INSERT INTO prompt_matching (source_id, target_id, model, is_match) VALUES (%s, %s, %s, %s)", (data[0], data[2], 'gpt-4o-mini', False))
    pg_conn.commit()

/var/folders/80/kl6rpl3d2xj20wv4hyb0dbgc0000gn/T/ipykernel_4071/1140915221.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  message = overal_chain({"sentence1": data[1], "sentence2": data[3]})
